In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Import datasets
data = pd.read_csv(r"C:\Users\enric\OneDrive\Desktop\Materie\Year2\Data Visualization\A2\covid-variants.csv")

continents = pd.read_csv(r"C:\Users\enric\OneDrive\Desktop\Materie\Year2\Data Visualization\A2\continents-according-to-our-world-in-data.csv")
continents = continents.drop(["Code", "Year"], axis=1).rename({"Entity":"Country"}, axis=1)

demographics = pd.read_csv(r"C:\Users\enric\OneDrive\Desktop\Materie\Year2\Data Visualization\A2\demographics.csv")
demographics = demographics[["Country", "Population", "Area (sq. mi.)", "Infant mortality (per 1000 births)", "GDP ($ per capita)", "Climate", "Deathrate"]]


In [ ]:
# Change country names for merging within datasets to be possible
continents["Country"] = continents["Country"].replace({"Kosovo":"Serbia", # aggregating cases for Kosovo, Montenegro and Serbia
                                                       "Montenegro":"Serbia",
                                                      "North Macedonia":"Macedonia",
                                                      "Curacao":"Netherlands Antilles", # aggregating cases for Curacao and Sint Maarten
                                                      "Sint Maarten (Dutch part)":"Netherlands Antilles"})

demographics["Country"] = demographics["Country"].str.rstrip()

demographics["Country"] = demographics["Country"].replace({"Bosnia & Herzegovina":"Bosnia and Herzegovina",
                                                            "Czech Republic":"Czechia",
                                                            "Gambia, The":"Gambia",
                                                            "Korea, South":"South Korea",
                                                            "Trinidad & Tobago":"Trinidad and Tobago"})


In [ ]:
# Filter datasets to only retain data up to December 2022
data.date = pd.to_datetime(data.date)
data = data[data.date.dt.year != 2022].sort_values("date")


In [ ]:
# Variants "others" and "non_who" are the same thing: we keep only one
data = data[data["variant"] != "non_who"]

In [ ]:
# Aggregate really similar variants with few cases each
# Total Cases are set to 0 for most of the variants in the same group, or aggregating you would be multiplying them
data["variant_grouped"] = data["variant"]

data.variant_grouped[data.variant == "B.1.1.277"] = "B.1.1"
data.num_sequences_total[data.variant == "B.1.1.302"] = 0
data.variant_grouped[data.variant == "B.1.1.302"] = "B.1.1"
data.num_sequences_total[data.variant == "B.1.1.519"] = 0
data.variant_grouped[data.variant == "B.1.1.519"] = "B.1.1"

data.variant_grouped[data.variant == "B.1.160"] = "B.1"
data.num_sequences_total[data.variant == "B.1.177"] = 0
data.variant_grouped[data.variant == "B.1.177"] = "B.1"
data.num_sequences_total[data.variant == "B.1.221"] = 0
data.variant_grouped[data.variant == "B.1.221"] = "B.1"
data.num_sequences_total[data.variant == "B.1.258"] = 0
data.variant_grouped[data.variant == "B.1.258"] = "B.1"
data.num_sequences_total[data.variant == "B.1.367"] = 0
data.variant_grouped[data.variant == "B.1.367"] = "B.1"
data.num_sequences_total[data.variant == "B.1.620"] = 0
data.variant_grouped[data.variant == "B.1.620"] = "B.1"

data.variant_grouped[data.variant == "S:677P.Pelican"] = "S:677"
data.num_sequences_total[data.variant == "S:677H.Robin1"] = 0
data.variant_grouped[data.variant == "S:677H.Robin1"] = "S:677"


In [ ]:
# A few slighly negative percentages are present in the data, we change those to zero
data.perc_sequences[data.perc_sequences < 0] 

In [ ]:
# Merge with continents and demographics dataset
data = data.groupby(by = ["date", "variant_grouped", "location"]).sum().reset_index()
data = pd.merge(data, continents, left_on = "location", right_on = "Country").drop("location", axis = 1)
data = pd.merge(data, demographics, left_on = "Country", right_on = "Country", how = "left")

In [ ]:
# A few interesting calculated fields
data["Cases per Capita"] = data.num_sequences / data.Population
data["Total Cases per Capita"] = data.num_sequences_total / data.Population

In [ ]:
# Format data from newly merged datasets
data["Infant mortality (per 1000 births)"] = data["Infant mortality (per 1000 births)"].str.replace(",",".").astype(float)
data["Deathrate"] = data["Deathrate"].fillna("0,0").astype(str).str.replace(",",".").astype(float)

In [ ]:
# Final check of column types
data.dtypes

In [ ]:
# Final check of missing values
data.isna().sum()


In [ ]:
# Export data to csv
data.to_csv("data.csv")